In [1]:
'''A ismple one-layer neural network'''
import torch 
x = torch.ones(5) #input tensor
y =  torch.zeros(3) #output

w = torch.randn(5, 3, requires_grad = True) #除了在创建tensor的时候设置requires_grad = True;还可以显示的
#通过w.required_grad_(True)来进行设置
b = torch.randn(3, requires_grad = True)

z = torch.matmul(x, w) + b

loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

# 1.Autograd自动微分
训练神经网络的时候，最常用的就是反向传播算法。在该算法中，参数（模型的权重）通过计算loss函数对于这些参数的梯度来进行更新。<br>
torch.autograd支持任何计算图的梯度自动计算。<br>

# 2.上方这个一层神经网络的计算图
![](./images/一层神经网络的计算图.jpg)
在这个计算图中，w和b都是需要优化的参数，因此我们需要计算loss函数对于它们的梯度。<br>

In [ ]:
# Note:
# 在构建计算图的时候，我们对于tensor使用的function，实际上是Function类的一个对象。这个对象知道在前向传播的时候
# 应该如何计算这个function，在后向传播的时候知道如何计算梯度。

# 反向传播的function可以通过tensor.grad_fn属性获取。前向传播的函数就是模型的结构。
print(z.grad_fn)
print(loss.grad_fn)


In [ ]:
'''Computing Gradients'''
# 为了优化神经网络的参数，我们需要计算loss函数对于这些参数的导数。
# 我们通常通过调用loss.backward()函数计算导数，并且使用.grad属性来计算导数的值。

loss.backward()
print(w.grad)
print(b.grad)

# 注意：
# （1）我们只能对计算图的中的叶子结点（也就是required_grad = True的那些结点）使用.grad属性获取梯度，对于其他图中的结点是不行的。
# （2）对于计算图，我们只能进行一次backward()操作来计算梯度，如果需要在一张图上进行多次backward操作，那么需要在调用的时候，设置retain_grah = True. loss.backward(retain_graph = True)


tensor([[0.1327, 0.2033, 0.3272],
        [0.1327, 0.2033, 0.3272],
        [0.1327, 0.2033, 0.3272],
        [0.1327, 0.2033, 0.3272],
        [0.1327, 0.2033, 0.3272]])
tensor([0.1327, 0.2033, 0.3272])


In [6]:
'''Disable Gradient Tracking'''
# 默认情况下，设置了requires_grad = True的tensor会追踪它们的计算历史并且支持梯度计算。
# 如果我们仅仅只想做前向计算，那么我们可以禁用追踪梯度的计算，具体实现有以下两种方式：
# 1.with torch.no_grad()
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

# 2.使用detach()
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

True
False
False


# 1.with torch.no_grad()和detach()的区别
![](./images/禁用梯度追踪的两种方法的区别.jpg)

# 关于计算图的理论
1.autograd会把数据（tensor）和所有执行过的操作（包括操作新生成的tensor）保存在一个有向无环图（directed acyclic graph, DAG）.
在DAG中，叶子是输入的tensor，根就是输出的tensor。通过从图的根结点到叶子结点，可以通过链式法则自动地计算梯度。<br>

2.在前向传播的过程中，autograd会同时完成以下操作：<br>
（1）运行forward()中定义的所有操作，得到最终的结果tensor<br>
（2）将操作的梯度函数(operation's gradient function)保留在DAG中。

3.反向传播的时候，当DAG的根结点调用.backward()调用的时候，会完成以下操作：<br>
（1）通过每个tensor的.grad_fn函数计算其梯度；<br>
（2）将每个tensor的梯度累积在它们的.grad属性中；<br>
（3）通过链式法则，一直传播直到叶子结点。<br>


4.DAG在pytorch中是动态的，每次都是重新创建的，在每次.backward()调用的时候，autograd会创建一个新的图。这种每次重新创建DAG的方式，也就允许了forward函数中的流程控制逻辑，每次迭代的时候都可以改变数据传播的方式。<br>



In [ ]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()

print(inp)
print(out)

# 如果不执行梯度清除，那么会不断累积
out.backward(torch.ones_like(out), retain_graph = True)
print(inp.grad)
out.backward(torch.ones_like(out), retain_graph = True)
print(inp.grad)

inp.grad.zero_() #正常在进行反向传播之前，都要先将梯度清除，实际
# 情况中，通常是optimizer来完成的
out.backward(torch.ones_like(out), retain_graph = True)
print(inp.grad)

# Note
# 如果直接调用.backward()而不带任何参数，那么等同于backward(torch.tensor(1.0))
# 通常是在一个scalar-valued function的情况下使用，比如网络训练最终的loss值。

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]], requires_grad=True)
tensor([[4., 1., 1., 1.],
        [1., 4., 1., 1.],
        [1., 1., 4., 1.],
        [1., 1., 1., 4.],
        [1., 1., 1., 1.]], grad_fn=<TBackward0>)
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


# 1.补充：Jackbian Products
![](./images/Jacobian.jpg)<br>
![](./images/Jacobian_1.jpg)<br>
![](./images/Jacobian_2.jpg)